# Import Librerie

In [ ]:
#@title Importa librerie
%reset -f
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import skimage.data as data
import scipy.ndimage as ndi
import tensorflow.keras
import pandas as pd
import os
from time import time
import shutil

In [ ]:
#@title Funzioni Utili
def convert_list_to_string(org_list, seperator=' '):
    return seperator.join(org_list)

def convert_string_to_list(string):
    li = list(string.split(" "))
    return li

# Caricamento Immagini in Collab

In [ ]:
#@title Download Immagini
!wget -q -c https://www.dropbox.com/s/44695urxpk4xc2r/img_align_celeba.zip?dl=0

In [ ]:
#@title Unzip Immagini
!unzip -q -n img_align_celeba.zip?dl=0

# Caricamento etichette immagini

In [ ]:
#@title Download Etichette Immagini
!wget -q -c https://www.dropbox.com/s/oolq770kd2wp18i/list_attr_celeba.txt?dl=0
fEtichette = open('list_attr_celeba.txt?dl=0',"r") #lettura da file 
listaEtichette = fEtichette.readlines() 
fEtichette.close()

In [ ]:
#@title Elaborazione Etichette Immagini

# Il file list_attr_celeba.txt contiene alla prima riga il numero di immagini prese in considerazione
numero_Immagini = int(listaEtichette[0])

# Dato che devo estrarre le etichette dal file, l'informazione di tale riga
# risulta essere superflua, e può essere pertanto cancellata. 
listaEtichette.pop(0)
attributi = listaEtichette[0]

# Salvo in memoria anche un array di attributi, mi serve dopo
arrayAttributi = convert_string_to_list(attributi)
arrayAttributi.pop(0) #Elimino Nome_Immagine dagli attributi
arrayAttributi.pop(len(arrayAttributi)-1) #Elimino \n, il carattere terminatore di stringa dall'array attributi

# Dato che ho salvato gli attributi, cancello l'informazione di tale riga
listaEtichette.pop(0)


In [ ]:
#@title Crezione DataFrame per CSV
trainPerCSV = []
validPerCSV = []
testPerCSV = []

trainPerCSV.append(attributi)
validPerCSV.append(attributi)
testPerCSV.append(attributi)

# Definisco la percentuale di dati che voglio assegnare a 
# training set, validation set e test set.
# % training set = splitSize*100 
# % validation = 10%
# % test set = 100 % - % training set - % validation

# In questo caso voglio assegnare 70% training dunque:
splitSize = 0.7

numeroImmaginiTrainSet = int( numero_Immagini* splitSize)
numeroImmaginiValidSet = int(numero_Immagini * 0.10) + 1
numeroImmaginiTestSet = int(numero_Immagini - numeroImmaginiTrainSet
                            - numeroImmaginiValidSet)

# Costruisco i file CSV contenenti le etichette per ogni immagine
# I dati iniziali sono classificati con etichette -1 e 1 (classificazione binaria)
# Utilizzando il metodo .replace() considero l'etichetta -1 come etichetta 0

for i in range(0,numeroImmaginiTrainSet ):
  line = listaEtichette[i].replace("-1","0")
  line = line.replace('  ', ' ')
  trainPerCSV.append(line)
for i in range(numeroImmaginiTrainSet, 
               numeroImmaginiTrainSet + numeroImmaginiValidSet ):
  line = listaEtichette[i].replace("-1","0")
  line = line.replace('  ', ' ')
  validPerCSV.append(line)
for i in range(numeroImmaginiTrainSet + numeroImmaginiValidSet, 
               numeroImmaginiTrainSet + numeroImmaginiValidSet 
               + numeroImmaginiTestSet):
  line = listaEtichette[i].replace("-1","0")
  line = line.replace('  ', ' ')
  testPerCSV.append(line)


# Creazione CSV

In [ ]:
#@title Creazione e salvataggio CSV
# Il comando np.savetxt salva i valori di una struttura dati in un file csv
np.savetxt("trainingSet.csv", trainPerCSV, delimiter=' ', fmt='%s')
np.savetxt("validationSet.csv", validPerCSV, delimiter=' ', fmt='%s')
np.savetxt("testSet.csv", testPerCSV, delimiter=' ', fmt='%s')

# Divisione Immagini in tre cartelle

In [ ]:
#@title Creazione cartelle per Immagini
try:
    os.mkdir('/content/training')
    os.mkdir('/content/validation')
    os.mkdir('/content/testing')
except OSError:
    pass

In [ ]:
#@title Funzione per la divisione in cartelle
from shutil import move
def split_data(SOURCE, TRAINING, VALIDATION, TESTING):
    files = []
    print('Split Data')
    for filename in os.listdir(SOURCE):
        file = SOURCE +'/'+ filename
        if os.path.getsize(file) > 0:
            files.append(filename)
        else:
            print(filename + " is zero length, so ignoring.")
    
    #per creare una divisione random
    #shuffled_set = random.sample(files, len(files))
    
    #per una divisione ordinata 
    files = np.sort(files)

    training_length = numeroImmaginiTrainSet
    validation_length = numeroImmaginiValidSet
    testing_length = numeroImmaginiTestSet

    training_set = files[0:training_length]
    validation_set = files[training_length:(training_length+validation_length)]
    testing_set = files[(training_length+validation_length):]

    for filename in training_set:
        this_file = SOURCE +'/'+ filename
        destination = TRAINING +'/'+ filename
        move(this_file, destination)
    
    for filename in validation_set:
        this_file = SOURCE +'/'+ filename
        destination = VALIDATION+'/' + filename
        move(this_file, destination)
        
    for filename in testing_set:
        this_file = SOURCE +'/'+ filename
        destination = TESTING+'/' + filename
        move(this_file, destination)

In [ ]:
#@title Divisione Immagini nelle cartelle
source_path= '/content/img_align_celeba'
train_dir_path = '/content/training'
valid_dir_path = '/content/validation'
test_dir_path = '/content/testing'
split_data(source_path, train_dir_path, valid_dir_path, test_dir_path)

# Caricamento dei dati in Keras

In [ ]:
#@title Creazione di g_test, g_train e g_valid
from tensorflow.keras.preprocessing.image import ImageDataGenerator

size = 320 # (size,size) è la dimensione finale delle immagini
numeroColonne = len(arrayAttributi) + 1

train_df = pd.read_csv('trainingSet.csv',delimiter=' ', usecols= np.arange(0,numeroColonne))
valid_df = pd.read_csv('validationSet.csv',delimiter=' ', usecols= np.arange(0,numeroColonne))
test_df = pd.read_csv('testSet.csv',delimiter=' ', usecols= np.arange(0,numeroColonne))

# Scelgo la class mode per l'estrazione dei dati dal csv
# Dato che voglio considerare un insieme di colonne, da cui prelevare le etichette, scelgo raw
class_mode = 'raw' 
gen_train = ImageDataGenerator(rescale = 1/255.0,
                               rotation_range= 10,
                               horizontal_flip=True,
                               vertical_flip=True,
                               width_shift_range= (size*5)//100, 
                               height_shift_range= (size*5)//100,
                               zoom_range= [0.9,1.1],
                               dtype=np.float32)
trainBatch = 3 
g_train = gen_train.flow_from_dataframe(dataframe=train_df,directory=train_dir_path, 
                                        target_size=(size,size),batch_size=trainBatch,
                                        class_mode=class_mode,
                                        x_col='Nome_Immagine',y_col=arrayAttributi,
                                        classes = arrayAttributi)
validBatch = 5 
gen_valid = ImageDataGenerator(rescale = 1/255.0,
                               dtype=np.float32)
g_valid = gen_valid.flow_from_dataframe(dataframe=valid_df,directory=valid_dir_path, 
                                        target_size=(size,size),batch_size=validBatch,
                                        class_mode=class_mode,
                                        x_col='Nome_Immagine',y_col=arrayAttributi,
                                        classes = arrayAttributi)
testBatch = 5 

gen_test = ImageDataGenerator(rescale = 1/255.0,
                              dtype=np.float32)
g_test = gen_test.flow_from_dataframe(dataframe=test_df,directory=test_dir_path, 
                                      target_size=(size,size),batch_size=testBatch,
                                      class_mode=class_mode,shuffle=False,
                                      x_col='Nome_Immagine',y_col=arrayAttributi,
                                      classes = arrayAttributi)


# Architettura

In [ ]:
#@title Creazione rete
from tensorflow.keras.applications import EfficientNetB3
import tensorflow.keras.layers as ly
from tensorflow.keras.models import Sequential 

base = EfficientNetB3(weights= 'imagenet',input_shape=(size,size,3),include_top=False)
print(len(base.layers))
net = Sequential()
net.add(base)
net.add(ly.GlobalAveragePooling2D())
net.add(ly.Dense(40,activation='sigmoid'))
layer_not_train = (len(base.layers)*25)//100
for layer in base.layers[0:layer_not_train]:
  layer.trainable = False
net.summary()

# Addestramento

In [ ]:
linkModelli = [
               'https://www.dropbox.com/s/zd7mr0i1ad6hvr4/Numero_Epoche_5_LearningRate_0.30000001192092896.zip?dl=0',
               'https://www.dropbox.com/s/00bg895r7nznia1/Numero_Epoche_10_LearningRate_0.30000001192092896.zip?dl=0',
               'https://www.dropbox.com/s/04o9pp9u3am87dd/Numero_Epoche_10_LearningRate_0.0010000000474974513.zip?dl=0',
               'https://www.dropbox.com/s/tzn3yl5qobxyse6/Numero_Epoche_10_LearningRate_0.00001.zip?dl=0',
               'https://www.dropbox.com/s/srbdze8ez5o0erg/Numero_Epoche_15_LearningRate_0.0010000000474974513.zip?dl=0',
               'https://www.dropbox.com/s/rype1u5odc4n63e/Numero_Epoche_20_LearningRate_0.009999999776482582.zip?dl=0',
               'https://www.dropbox.com/s/rgjnd4j37dcra5i/Numero_Epoche_20_LearningRate_0.005.zip?dl=0'
]

In [ ]:
#@title Funzione per ottenre il L.R. e le Epoche dal modello pre-allenato
def getValueFromModel(link):
  lr = 0
  epoche = 0
  link = link.replace('https://www.dropbox.com/s/','')
  index_to_remove = link.find('/') +1
  link = link[index_to_remove:]
  link = link.replace('Numero_Epoche_','')
  index_to_remove = link.find('_Lea')
  epoche = int(link[:index_to_remove])
  link = link.replace(str(epoche),'')
  link = link.replace('_LearningRate_0.','')
  link = link.replace('.zip?dl=0','')
  lr = '0.' + link
  lr = float(lr)
  return lr, epoche

In [ ]:
def remove_exponent(value):
    decial = value.split('e')
    ret_val = format(((float(decial[0]))*(10**int(decial[1]))), '.8f')
    return ret_val

In [ ]:
#@title Scegli i valori di L.R, epoche 
i = 0

for link in linkModelli:
  i += 1
  lr, numEpochs = getValueFromModel(link)
  if lr <= 0.0001: 
    lrt = remove_exponent(str(lr))
    index_to_remove = lrt.find('1') +1
    lrt = lrt[:index_to_remove]
  print('Clicca ',i,' per il modello con learning rate: ', lr, ' ed numero di epoche: ', numEpochs)
print('Clicca ', len(linkModelli)+1, ' per creare un nuovo modello')
scelta = 0
while (scelta == 0):
  try:
    scelta = int(input('Numero di modello scelto '))
  except:
    print('Scelta errata')
if scelta <= len(linkModelli):
  linkScelto = linkModelli[scelta-1]
  lr, numEpochs = getValueFromModel(linkScelto)
  if lr >= 0.0001:
    model_save_name = 'Numero_Epoche_' + str(numEpochs) + '_LearningRate_'+ str(lr)
  else:
    model_save_name = 'Numero_Epoche_' + str(numEpochs) + '_LearningRate_'+ lrt
  !wget -q -c {linkScelto}
else:
  numEpochs = int(input('Numero di epoche '))
  try:
    lr = float(input('Learnig rate (0.xxxxxxxx)'))
    model_save_name = 'Numero_Epoche_' + str(numEpochs) + '_LearningRate_'+ str(remove_exponent(lr))
  except:
    print('lr non è dell formato giusto \n Deve essere un deciamle 0.xxxx')

In [ ]:
#@title Compile rete
from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate=lr, 
                 epsilon = 1e-08)
net.compile(loss='binary_crossentropy',optimizer=optimizer,
            metrics=['binary_accuracy'])
immTrainSet = numeroImmaginiTrainSet//numEpochs
immValidSet = numeroImmaginiValidSet//numEpochs

In [ ]:
#@title Carica modello preesistente se esiste
path_file = model_save_name+'.zip?dl=0'
if os.path.exists(path_file):
  shutil.unpack_archive(path_file,format='zip',
                        extract_dir=('/content/'+model_save_name))
  #os.remove('/content/'+path_file)
  net = tensorflow.keras.models.load_model(model_save_name,compile=True)
  net.summary()
else:
  print('rete non trovata, alleno la rete con i parametri indicati')
  time_before = int(time())//60
  history = net.fit(g_train,steps_per_epoch=immTrainSet,epochs=numEpochs,
                  verbose=True, validation_data=g_valid,
                  validation_steps=immValidSet)

  time_after = int(time())//60
  deltaT = time_after - time_before # In minuti
  print(deltaT , ' minuti')

# Valutazione delle prestazioni

In [ ]:
#@title Stampa caratteristiche riconosciute
def print_acc(matrixAccuracy,soglia = 0):
  print("Caratteristiche dell'immagine:")
  for i in range(len(arrayAttributi)):
    if matrixAccuracy[i][1] >= soglia:
      if matrixAccuracy[i][2] == 1:
        print(matrixAccuracy[i][0] ,' => ' , matrixAccuracy[i][1], '% | expected : Ha l\'attributo')
      else:
        print(matrixAccuracy[i][0] ,' => ' , matrixAccuracy[i][1], '% | expected : Non ha l\'attributo')

In [ ]:
#@title Test con un'immagine casuale
import cv2

nomeImmagine = test_df['Nome_Immagine'][np.random.randint(0,len(test_df['Nome_Immagine'])-1)]
path = '/content/testing/' + nomeImmagine

test_image = np.float32(io.imread(path))
test_image = cv2.resize(test_image,dsize=(size,size), interpolation=cv2.INTER_CUBIC)/255
plt.figure();
plt.imshow(test_image,clim=[0,1]);
plt.show()

test_image = np.reshape(test_image,(-1,size,size,3))
pred = net.predict(test_image)[0]

#creo una matrice dove per ogni riga ho un attributo con la relativa accuracy dell'immagine

matrixAccuracy = []
indexImage = 0
for i in range(len(test_df)):
  if test_df['Nome_Immagine'][i] == nomeImmagine:
    indexImage = i
    break

expected = test_df.iloc[indexImage,1:41].copy()
for i in range(len(arrayAttributi)):
  matrixAccuracy.append([arrayAttributi[i],round(pred[i]*100,2),expected[i]])

#Stampo tutti gli attributi che hanno valore maggiore di soglia%
soglia = 0
print_acc(matrixAccuracy,soglia)

In [ ]:
def print_scores(scores):
  print('La performance dei sigoli attributi è: \n')
  for score in scores:
    print(' ',score[0],': ',score[1], '% \n')

In [ ]:
def getLayerNonAllenati(net):
  result = 0
  for layer in net.layers:
    if layer.trainable == False:
      result += 1
  return result

In [ ]:
#@title Calcolo dell'AUC per tutto il testSet
from sklearn.metrics import roc_auc_score

# Costruisco un array di predizioni predArray
predArray = net.predict(g_test,verbose=True) 
# Considerando l'array di valori corretti e predArray quello di valori stimati
valCorrettiTestArray = test_df.iloc[:,1:numeroColonne].copy()

# Calcolo accuracy in termini di auc per ogni singolo attributo
aucArray = []
for i in range(1,numeroColonne):
  valCorrettiTestArray = test_df.iloc[:,i:i+1].copy() # Considero solo la componente dell'attributo i-esimo 
  auc = roc_auc_score(y_true=valCorrettiTestArray,y_score=predArray[:,i-1],average='macro')
  #Costruisco una matrice avente due colonne: 1 per gli attributi e 1 per la relativa AUC
  aucArray.append([arrayAttributi[i-1], round((auc)*100,2)]) 

layer_non_allenati = getLayerNonAllenati(net)
print('I numero di layer della rete non allenati è: ',layer_non_allenati,'\n')

print_scores(aucArray)

